In [224]:
import import_ipynb
import openai
from dotenv import load_dotenv
import requests
import re
from bs4 import BeautifulSoup
import os
from datetime import datetime, timedelta
import locale
import unicodedata

In [225]:
%run "../jgc_news/g1.ipynb" import G1
%run "../jgc_news/dn.ipynb" import DN
%run "../jgc_news/odia.ipynb" import ODia
%run "../jgc_news/estadao.ipynb" import Estadao
%run "../jgc_news/folha.ipynb" import Folha

load_dotenv()
client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

In [226]:
def str_cleasing(str):
    str = re.sub(r'\n', ' ', str)
    str = re.sub(r'\s+', ' ', str)
    str = re.sub(r'\s+$', ' ', str)
    str = re.sub(r'^\s+', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    str = re.sub(r'  ', ' ', str)
    return str

In [227]:
def normalize_string(text):
    normalized_text = unicodedata.normalize('NFKD', text)
    text_without_accents = ''.join([c for c in normalized_text if not unicodedata.combining(c)])
    final_text = text_without_accents.lower().replace(' ', '-')
    return final_text

In [228]:
times = [
    {
        "timeName": 'Athletico-PR', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/pr/futebol/times/athletico-pr/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/atletico-paranaense/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/atletico-pr/"}
        ]}, {
        "timeName": 'Atlético-GO', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/go/futebol/times/atletico-go/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/atletico-clube-goianiense/"}
        ]},{
        "timeName": 'Atlético-MG', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/atletico-mg/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/atletico-mineiro/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/atletico-mg/"}
        ]},{
        "timeName": 'Bahia', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/ba/futebol/times/bahia/"}
        ]},{
        "timeName": 'Botafogo', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/botafogo/"},
            {"source": "odia", "url": "https://odia.ig.com.br/esporte/botafogo"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/botafogo/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/botafogo/"}
        ]},{
        "timeName": 'Bragantino', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/vale-do-paraiba-regiao/futebol/times/bragantino/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/bragantino/"}
        ]},{
        "timeName": 'Corinthians', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/corinthians/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/corinthians/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/corinthians/"}
        ]},{
        "timeName": 'Criciúma', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sc/futebol/times/criciuma/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/criciuma-esporte-clube/"}
        ]},{
        "timeName": 'Cruzeiro', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/cruzeiro/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/cruzeiro-esporte-clube/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/cruzeiro/"}
        ]},{
        "timeName": 'Cuiaba', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/mt/futebol/times/cuiaba/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/cuiaba-esporte-clube/"}
        ]},{
        "timeName": 'Flamengo', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/flamengo/"},
            {"source": "odia", "url": "https://odia.ig.com.br/esporte/flamengo"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/flamengo/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/flamengo/"}
        ]},{
        "timeName": 'Fluminense', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/fluminense/"},
            {"source": "odia", "url": "https://odia.ig.com.br/esporte/fluminense"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/fluminense/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/fluminense/"}
        ]},
        {
        "timeName": 'Fortaleza', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/ce/futebol/times/fortaleza/"},
            {"source": "dn", "url": "https://diariodonordeste.verdesmares.com.br/noticias/sobre/Esportes-fortaleza%20esporte%20clube"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/https://www.estadao.com.br/tudo-sobre/fortaleza-esporte-clube/"}
        ]}
        ,{
        "timeName": 'Grêmio', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/rs/futebol/times/gremio/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/gremio/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/gremio/"}
        ]},{
        "timeName": 'Internacional', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/rs/futebol/times/internacional/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/internacional/"}
        ]},{
        "timeName": 'Juventude', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/rs/futebol/times/juventude/"}
        ]},{
        "timeName": 'Palmeiras', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/palmeiras/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/palmeiras/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/palmeiras/"}
        ]},{
        "timeName": 'São-Paulo', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/sao-paulo/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/sao-paulo-futebol-clube/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/sao-paulo-fc/"}
        ]},{
        "timeName": 'Vasco', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/vasco/"},
            {"source": "odia", "url": "https://odia.ig.com.br/esporte/vasco"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/vasco/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/vasco/"}
        ]},{
        "timeName": 'Vitória', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/ba/futebol/times/vitoria/"}
        ]},{
        "timeName": 'Amazonas-FC', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/am/futebol/times/amazonas-fc/"}
        ]},{
        "timeName": 'América-MG', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/futebol/times/america-mg/"}
        ]},{
        "timeName": 'Avaí', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sc/futebol/times/avai/"}
        ]},{
        "timeName": 'Botafogo-SP', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/ribeirao-preto-e-regiao/futebol/times/botafogo-sp/"}
        ]},{
        "timeName": 'Brusque', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sc/futebol/times/brusque/"}
        ]},{
        "timeName": 'Ceará', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/ce/futebol/times/ceara/"},
            {"source": "dn", "url": "https://diariodonordeste.verdesmares.com.br/noticias/sobre/Esportes-cear%C3%A1%20sporting%20club"}
        ]},{
        "timeName": 'Chapecoense', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sc/futebol/times/chapecoense/"}
        ]},{
        "timeName": 'Coritiba', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/pr/futebol/times/coritiba/"},
            {"source": "folha", "url": "https://www1.folha.uol.com.br/folha-topicos/coritiba/"}
        ]},{
        "timeName": 'CRB', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/al/futebol/times/crb/"}
        ]},{
        "timeName": 'Goiás', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/go/futebol/times/goias/"}
        ]},{
        "timeName": 'Guarani', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/campinas-e-regiao/futebol/times/guarani/"}
        ]},{
        "timeName": 'Ituano', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/tem-esporte/futebol/times/ituano/"},
            {"source": "estadao", "url": "https://www.estadao.com.br/tudo-sobre/ituano/"}
        ]},{
        "timeName": 'Mirassol', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/tem-esporte/futebol/times/mirassol/"}
        ]},{
        "timeName": 'Novorizontino', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/tem-esporte/futebol/times/novorizontino/"}
        ]},{
        "timeName": 'Operário-PR', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/pr/futebol/times/operario-pr/"}
        ]},{
        "timeName": 'Paysandu', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/pa/futebol/times/paysandu/"}
        ]},{
        "timeName": 'Ponte-Preta', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/campinas-e-regiao/futebol/times/ponte-preta/"}
        ]},{
        "timeName": 'Santos', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/sp/santos-e-regiao/futebol/times/santos/"}
        ]},{
        "timeName": 'Sport', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/pe/futebol/times/sport/"}
        ]},{
        "timeName": 'Vila-Nova', 
        "sources": [
            {"source": "g1", "url": "https://ge.globo.com/go/futebol/times/vila-nova/"}
        ]},
    ]



In [229]:
allsources = []
for team in times:
    newsRefereces = []
    for source in team['sources']:
        if source['source'] == 'g1':
            g1 = G1()
            urls = g1.getUrls(source['url'])
            for url in urls:
                newsData = g1.getNewsData(url)
                newsRefereces.append(newsData)
        elif source['source'] == 'dn':
            dn = DN()
            urls = dn. getUrls(source['url'], normalize_string(team['timeName']))
            for url in urls:
                newsData = dn.getNewsData(url)
                newsRefereces.append(newsData)
        elif source['source'] == 'odia':
            odia = ODia()
            urls = odia. getUrls(source['url'], normalize_string(team['timeName']))
            for url in urls:
                newsData = odia.getNewsData(url)
                newsRefereces.append(newsData)
        elif source['source'] == 'estadao':
            estadao = Estadao()
            urls = estadao. getUrls(source['url'])
            for url in urls:
                newsData = estadao.getNewsData(url)
                newsRefereces.append(newsData)
        elif source['source'] == 'folha':
            folha = Folha()
            urls = folha. getUrls(source['url'])
            for url in urls:
                newsData = folha.getNewsData(url)
                newsRefereces.append(newsData)
        else:
            print('Fonte não encontrada')
    newsRefereces = [x for x in newsRefereces if x is not None]
    allsources.append({"time": team['timeName'], "news": newsRefereces})
    print('Time:', team['timeName'], '-', len(newsRefereces), 'notícias coletadas')


Time: Athletico-PR - 1 notícias coletadas
Time: Atlético-GO - 3 notícias coletadas
Time: Atlético-MG - 0 notícias coletadas
Time: Bahia - 2 notícias coletadas
Time: Botafogo - 9 notícias coletadas
Time: Bragantino - 0 notícias coletadas
Time: Corinthians - 4 notícias coletadas
Time: Criciúma - 0 notícias coletadas
Time: Cruzeiro - 0 notícias coletadas
Time: Cuiaba - 0 notícias coletadas
Time: Flamengo - 4 notícias coletadas
Time: Fluminense - 11 notícias coletadas
Time: Fortaleza - 11 notícias coletadas
Time: Grêmio - 0 notícias coletadas
Time: Internacional - 3 notícias coletadas
Time: Juventude - 4 notícias coletadas
Time: Palmeiras - 4 notícias coletadas
Time: São-Paulo - 5 notícias coletadas
Time: Vasco - 8 notícias coletadas
Time: Vitória - 4 notícias coletadas
Time: Amazonas-FC - 1 notícias coletadas
Time: América-MG - 1 notícias coletadas
Time: Avaí - 2 notícias coletadas
Time: Botafogo-SP - 2 notícias coletadas
Time: Brusque - 0 notícias coletadas
Time: Ceará - 4 notícias colet

In [230]:
# folha = Folha()
# urls = folha.getUrls('https://www1.folha.uol.com.br/folha-topicos/flamengo/')
# for u in urls:
#     folha.getNewsData(u)
